In [380]:
import numpy as np
from fixedpoint import FixedPoint

In [381]:
'''cb8e35d2
9d89d848
537156d9
57b7d22b
72b616ec
6bd8bb20
e5ee447c
e2690140'''

'cb8e35d2\n9d89d848\n537156d9\n57b7d22b\n72b616ec\n6bd8bb20\ne5ee447c\ne2690140'

In [382]:
x_real = ['0x'+'0cb8e35d2', '0x' + '0537156d9', '0x' + '072b616ec', '0x' + '0e5ee447c']
x_imag = ['0x'+'09d89d848', '0x' + '057b7d22b', '0x' + '06bd8bb20', '0x' + '0e2690140']

In [383]:
def fixed_point(bits, signed=1, m=17, n=16):
    return float(FixedPoint(bits, signed, m, n))

In [384]:
x_real_decimal = list(map(fixed_point, x_real))
x_imag_decimal = list(map(fixed_point, x_imag))

In [385]:
list(zip(x_real_decimal, x_imag_decimal))

[(52110.2102355957, 40329.84484863281),
 (21361.339248657227, 22455.82096862793),
 (29366.08953857422, 27608.73095703125),
 (58862.267517089844, 57961.0048828125)]

In [386]:
signed, m, n = 1, 16, 16
bits = "0x2E118364"
x = FixedPoint(bits, signed, m, n)

float(x), x.qformat

(11793.513244628906, 'Q16.16')

In [387]:
def butterfly(xev_r, xev_i, xod_r, xod_i, N):

    k = np.arange(0, N/2)

    wr = np.cos(-2*np.pi*k/N)
    wi = np.sin(-2*np.pi*k/N)

    g1 = wr*xod_r - wi*xod_i
    g2 = wr*xod_i - wi*xod_r

    y_re = np.concatenate((xev_r + g1, xev_r - g1))
    y_im = np.concatenate((xev_i + g2, xev_i - g2))

    return y_re, y_im

def matrix_computation(x_re, x_im):
    N = len(x_re)

    k = np.arange(0, N)
    
    kj = np.kron(k, k)

    F_re = np.cos(-2*np.pi * kj / N)
    F_im = np.sin(-2*np.pi * kj / N)

    y_re = np.sum(F_re*x_re - F_im*x_im)
    y_im = np.sum(F_re*x_im - F_im*x_re)

    return y_re, y_im

def FFT(x_re, x_im):
    N = len(x_re)
    
    if N % 2 == 0:
        xev_r = x_re[::2]
        xev_i = x_im[::2]

        xod_r = x_re[1::2]
        xod_i = x_im[1::2]

        Xev_r, Xev_i = FFT(xev_r, xev_i)
        Xod_r, Xod_i = FFT(xod_r, xod_i)

        yr, yi = butterfly(Xev_r, Xev_i, Xod_r, Xod_i, N)
    else:
        yr, yi = matrix_computation(x_re, x_im)

    return yr, yi

def iFFT(x_re, x_im):
    N = len(x_re)
    Xre, Xim    = FFT(x_re, x_im)

    return Xre/N, Xim/N

In [395]:
y_r, y_i = FFT(x_real_decimal, x_imag_decimal)

In [389]:
def fixed_point_ftoh(float_nm, signed=1, m=20, n=16):
    return str(FixedPoint(float_nm, signed, m, n, str_base=16))

In [390]:
list(zip(y_r, y_i))

[(161699.906539917, 148355.4016571045),
 (-12761.063217163086, -24779.814376831055),
 (1252.6930084228516, -12478.250045776367),
 (58249.304611206055, 50222.04216003418)]

In [391]:
y_real_hex = list(map(fixed_point_ftoh, y_r))
y_imag_hex = list(map(fixed_point_ftoh, y_i))

In [392]:
y_real_hex

['277a3e813', 'fce26efd1', '004e4b169', '0e3894dfb']

In [393]:
y_imag_hex

['2438366d3', 'f9f342f85', 'fcf41bffd', '0c42e0acb']